### General API call flow

#### Steps

1. Producer = Prepare API request
   1. Get template from folder
   2. Render variables
      1. perPage = 10 (for traffic handling purpose)
   3. Check limit before proceed
      1. Use time aware queing
2. Consumer = Response parser and storage
   1. Parse `data` key
   2. Set storage destinationjson -> 1 json per api call

In [6]:

from pathlib import Path
import aiohttp
import asyncio
import json

In [7]:
async def make_api_call(url, query, variables):
    async with aiohttp.ClientSession() as session:
        async with session.post(url, json={'query': query, 'variables': variables}) as resp:
            result = await resp.json()
            header = resp.headers
            return result, header
        
async def producer_work(queue, current_page, url, query, per_page):

    variables = {  
        'page': current_page,
        'perPage': per_page
    }
    print(f"Requesting page {current_page} at {url}")

    result, header = await make_api_call(url, query, variables=variables)
    remaining_request = int(header.get('x-ratelimit-remaining'))
    print(f'Remaing API call per min: {remaining_request}')

    await queue.put(result)
    return result, header


In [8]:
async def save_api_to_local(file_path:str, data: dict):
    print(f"Writing to {file_path}")
    with open(file_path, 'w') as fp:
        json.dump(data, fp)

async def consumer_work(queue: asyncio.Queue, storage_path:Path):

    while True:

        response_body = await queue.get()
        data = response_body.get('data')
        current_page = response_body.get('data').get('Page').get('pageInfo').get('currentPage')

        file_path = Path(f'page-{current_page}.json')
        download_path = storage_path / file_path

        await save_api_to_local(file_path=str(download_path), data=data)

In [10]:
# async def main():
MAX_PER_PAGE = 50
TOP = 1000
MAX_API_PER_MIN = 90
total_api_call_count = int(TOP / MAX_PER_PAGE)

template_path = Path(r'graphql-template\get-anime.graphql')
with open(str(template_path), 'r') as fp:
    template = fp.read()

url = 'https://graphql.anilist.co'

base_path = Path().cwd().parent
storage_path = base_path / Path('raw/anime')

queue = asyncio.Queue()


In [11]:
producer_coros = [producer_work(queue, page_num, url, template, MAX_PER_PAGE)
    for page_num in range(6, total_api_call_count+1)]

In [12]:
# Start work by creating task from initialized coroutines
ress = [asyncio.create_task(coro) for coro in producer_coros]

Requesting page 6 at https://graphql.anilist.co
Requesting page 7 at https://graphql.anilist.co
Requesting page 8 at https://graphql.anilist.co
Requesting page 9 at https://graphql.anilist.co
Requesting page 10 at https://graphql.anilist.co
Requesting page 11 at https://graphql.anilist.co
Requesting page 12 at https://graphql.anilist.co
Requesting page 13 at https://graphql.anilist.co
Requesting page 14 at https://graphql.anilist.co
Requesting page 15 at https://graphql.anilist.co
Requesting page 16 at https://graphql.anilist.co
Requesting page 17 at https://graphql.anilist.co
Requesting page 18 at https://graphql.anilist.co
Requesting page 19 at https://graphql.anilist.co
Requesting page 20 at https://graphql.anilist.co


In [13]:
queue.qsize()

0

In [14]:
consumer_coros = [consumer_work(queue, storage_path) for _ in range(3)]

Remaing API call per min: 89
Remaing API call per min: 81
Remaing API call per min: 88
Remaing API call per min: 87
Remaing API call per min: 86
Remaing API call per min: 85
Remaing API call per min: 75
Remaing API call per min: 77
Remaing API call per min: 83
Remaing API call per min: 80
Remaing API call per min: 79
Remaing API call per min: 76
Remaing API call per min: 84
Remaing API call per min: 82
Remaing API call per min: 78


In [15]:
consumer_responses = [asyncio.create_task(coro) for coro in consumer_coros]

Writing to c:\Users\JH\project\road-to-data-engineer\graph-search\graph-search-data-engineering\raw\anime\page-11.json
Writing to c:\Users\JH\project\road-to-data-engineer\graph-search\graph-search-data-engineering\raw\anime\page-6.json
Writing to c:\Users\JH\project\road-to-data-engineer\graph-search\graph-search-data-engineering\raw\anime\page-19.json
Writing to c:\Users\JH\project\road-to-data-engineer\graph-search\graph-search-data-engineering\raw\anime\page-14.json
Writing to c:\Users\JH\project\road-to-data-engineer\graph-search\graph-search-data-engineering\raw\anime\page-7.json
Writing to c:\Users\JH\project\road-to-data-engineer\graph-search\graph-search-data-engineering\raw\anime\page-18.json
Writing to c:\Users\JH\project\road-to-data-engineer\graph-search\graph-search-data-engineering\raw\anime\page-8.json
Writing to c:\Users\JH\project\road-to-data-engineer\graph-search\graph-search-data-engineering\raw\anime\page-12.json
Writing to c:\Users\JH\project\road-to-data-enginee

In [97]:
await queue.join()

In [ ]:
queue.empty()